In [ ]:
import torch
import numpy as np
from imageio.v3 import imread
from FastGeodis import generalised_geodesic2d
import matplotlib.pyplot as plt

In [ ]:
assert torch.cuda.is_available()

In [ ]:
INPUT_IMG_FILENAME = ""
INPUT_MASK_FILENAME = ""

In [ ]:
if len(INPUT_IMG_FILENAME) > 0:
    img = torch.from_numpy(imread(INPUT_IMG_FILENAME).astype(np.float32)).unsqueeze_(0).unsqueeze_(0)
else:
    img = torch.zeros((1, 1, 40, 50))
mask = None
if len(INPUT_MASK_FILENAME) > 0:
    mask = torch.from_numpy((imread(INPUT_MASK_FILENAME) > 0).astype(np.uint8))
else:
    mask = torch.ones_like(img)
    mask[..., 20:30, 20:43] = 0
assert(img.shape == mask.shape)

In [ ]:
img = img.to("cuda")
mask = mask.to("cuda")

In [ ]:
v = 1e10
lamb = 0.0 # Geodesic distance
iterations = 100

In [ ]:
geo_dist = generalised_geodesic2d(img, mask, v, lamb, iterations)

In [ ]:
geo_dist = np.squeeze(geo_dist.cpu().numpy())

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(geo_dist, cmap="inferno")
plt.show()